In [3]:
import json
import torch
from transformers import BertTokenizer, BertForMultipleChoice
import os

# Load tokenizer and model
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMultipleChoice.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def predict_choice(prompt, options):
    # Create 4 choices with (prompt, option) pairs
    encoding = tokenizer(
        [prompt] * 4,
        options,
        return_tensors="pt",
        padding=True,
        truncation=True
    )
    input_ids = encoding['input_ids'].unsqueeze(0).to(device)
    attention_mask = encoding['attention_mask'].unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1).item()
        return ["A", "B", "C", "D"][prediction]

def process_json_mbert(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    output_path = json_path.replace(".json", "_mbert_output.json")
    if os.path.exists(output_path):
        with open(output_path, "r", encoding="utf-8") as f:
            results = json.load(f)
        start_index = len(results)
    else:
        results = []
        start_index = 0

    print(f"📝 Starting from item {start_index} for {json_path}")

    for i in range(start_index, len(data)):
        item = data[i]
        prompt = item["prompt"]
        options = [
            item["options"]["A"],
            item["options"]["B"],
            item["options"]["C"],
            item["options"]["D"]
        ]

        try:
            choice = predict_choice(prompt, options)
        except Exception as e:
            print(f"Error on item {i}: {e}")
            choice = "Error"

        item["mBERT Choice"] = choice
        results.append(item)

        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        print(f"✅ [{i+1}/{len(data)}] Saved choice: {choice}")

    print(f"✅ Completed: {output_path}")
    return results

# Run for all 3 JSON prompt files
for file in [
    "multiple_choice_prompts_Low.json",
    "multiple_choice_prompts_Mid.json",
    "multiple_choice_prompts_High.json"
]:
    process_json_mbert(file)



Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📝 Starting from item 0 for multiple_choice_prompts_Low.json
✅ [1/60] Saved choice: B
✅ [2/60] Saved choice: D
✅ [3/60] Saved choice: D
✅ [4/60] Saved choice: D
✅ [5/60] Saved choice: B
✅ [6/60] Saved choice: C
✅ [7/60] Saved choice: B
✅ [8/60] Saved choice: B
✅ [9/60] Saved choice: C
✅ [10/60] Saved choice: B
✅ [11/60] Saved choice: B
✅ [12/60] Saved choice: A
✅ [13/60] Saved choice: C
✅ [14/60] Saved choice: A
✅ [15/60] Saved choice: C
✅ [16/60] Saved choice: B
✅ [17/60] Saved choice: A
✅ [18/60] Saved choice: B
✅ [19/60] Saved choice: C
✅ [20/60] Saved choice: B
✅ [21/60] Saved choice: D
✅ [22/60] Saved choice: A
✅ [23/60] Saved choice: D
✅ [24/60] Saved choice: B
✅ [25/60] Saved choice: A
✅ [26/60] Saved choice: B
✅ [27/60] Saved choice: D
✅ [28/60] Saved choice: D
✅ [29/60] Saved choice: D
✅ [30/60] Saved choice: B
✅ [31/60] Saved choice: D
✅ [32/60] Saved choice: C
✅ [33/60] Saved choice: D
✅ [34/60] Saved choice: D
✅ [35/60] Saved choice: C
✅ [36/60] Saved choice: B
✅ [37/60] Sav